<a href="https://colab.research.google.com/github/Fyzie/DLIVA_CV_Workshop_Assignments/blob/main/assessment/G2110445_Assessment_DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assessment 1: Deep Learning

1) Answer all questions.
2) This assessment is open-book. You are allowed to refer to any references including online materials, books, notes, codes, github links, etc.
3) Copy this notebook to your google drive (click **FILE** > **save a copy in Drive**)
4) Upload the answer notebook to your github. 
5) Submit the assessment by sharing the link to your answer notebook. 





**QUESTION 1** 

One day while wandering around a clothing store at KL East Mall, you stumbled upon a pretty girl who is choosing a dress for Hari Raya. It turns out that the girl is visually impaired and had a hard time distinguishing between an abaya and a kebaya. To help people with the similar situation, you then decided to develop an AI system to identify the type of clothes using a Convolutional Neural Networks (ConvNet). In order to train the network, you decide to use the Fashion MNIST dataset which is freely available on Pytorch.


a) Given the problem, what is the most appropriate loss function to use? Justify your answer. **[5 marks]**


<span style="color:blue">
    ANSWER: CrossEntropyLoss, because it minimizes difference of two probabilities, predicted and actual. The probability of predictions should be either be 0 or 1. Compared to mean square loss, it is more suitable for regression or estimation problems. </span>

b) Create and train a ConvNet corresponding to the following CNN architecture (with a modification of the final layer to address the number of classes). Please include **[10 marks]**:

    1) The dataloader to load the train and test datasets.

    2) The model definition (either using sequential method OR pytorch class method).

    3) Define your training loop.

    4) Output the mean accuracy for the whole testing dataset.

    

<div>
<img src="https://vitalflux.com/wp-content/uploads/2021/11/VGG16-CNN-Architecture.png" width="550"/>
</div>


In [1]:
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import glob
import numpy
import random

from PIL import Image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from torchsummary import summary

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize((100,100)),
     transforms.Lambda(lambda x: x.repeat(3,1,1) if x.size(0)==1 else x),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [3]:
# batch_size
batch_size = 16

# datasets
trainset = torchvision.datasets.FashionMNIST('./data',
    download=True,
    train=True,
    transform=transform)
testset = torchvision.datasets.FashionMNIST('./data',
    download=True,
    train=False,
    transform=transform)

# dataloaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                        shuffle=True, num_workers=2)


testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                        shuffle=False, num_workers=2)

# constant for classes
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

train_data_size = len(trainloader.dataset)
test_data_size = len(testloader.dataset)

print(train_data_size)
print(test_data_size)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw

60000
10000


In [28]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, 3)
        self.conv2 = nn.Conv2d(8, 16, 3)
        self.conv3 = nn.Conv2d(16, 32, 3)
        self.conv4 = nn.Conv2d(32,64, 3)
        self.conv5 = nn.Conv2d(64, 128, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 1 * 1, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = self.pool(self.relu(self.conv4(x)))
        x = self.pool(self.relu(self.conv5(x)))
        # print(x.shape)
        x = x.view(-1, 128 * 1 * 1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [29]:
model = CNN() # need to instantiate the network to be used in instance method

# 2. LOSS AND OPTIMIZER
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 3. move the model to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)
summary(model, (3, 100, 100))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 98, 98]             224
              ReLU-2            [-1, 8, 98, 98]               0
         MaxPool2d-3            [-1, 8, 49, 49]               0
            Conv2d-4           [-1, 16, 47, 47]           1,168
              ReLU-5           [-1, 16, 47, 47]               0
         MaxPool2d-6           [-1, 16, 23, 23]               0
            Conv2d-7           [-1, 32, 21, 21]           4,640
              ReLU-8           [-1, 32, 21, 21]               0
         MaxPool2d-9           [-1, 32, 10, 10]               0
           Conv2d-10             [-1, 64, 8, 8]          18,496
             ReLU-11             [-1, 64, 8, 8]               0
        MaxPool2d-12             [-1, 64, 4, 4]               0
           Conv2d-13            [-1, 128, 2, 2]          73,856
             ReLU-14            [-1, 12

In [4]:
import time # to calculate training time

def train_and_validate(model, loss_criterion, optimizer, epochs=25):
    '''
    Function to train and validate
    Parameters
        :param model: Model to train and validate
        :param loss_criterion: Loss Criterion to minimize
        :param optimizer: Optimizer for computing gradients
        :param epochs: Number of epochs (default=25)
  
    Returns
        model: Trained Model with best validation accuracy
        history: (dict object): Having training loss, accuracy and validation loss, accuracy
    '''
    
    start = time.time()
    history = []
    best_acc = 0.0

    for epoch in range(epochs):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch+1, epochs))
        
        # Set to training mode
        model.train()
        
        # Loss and Accuracy within the epoch
        train_loss = 0.0
        train_acc = 0.0
        
        valid_loss = 0.0
        valid_acc = 0.0
        
        for i, (inputs, labels) in enumerate(trainloader):

            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Clean existing gradients
            optimizer.zero_grad()
            
            # Forward pass - compute outputs on input data using the model
            outputs = model(inputs)
            
            # Compute loss
            loss = loss_criterion(outputs, labels)
            
            # Backpropagate the gradients
            loss.backward()
            
            # Update the parameters
            optimizer.step()
            
            # Compute the total loss for the batch and add it to train_loss
            train_loss += loss.item() * inputs.size(0)
            
            # Compute the accuracy
            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))
            
            # Convert correct_counts to float and then compute the mean
            acc = torch.mean(correct_counts.type(torch.FloatTensor))
            
            # Compute total accuracy in the whole batch and add to train_acc
            train_acc += acc.item() * inputs.size(0)
            
            #print("Batch number: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}".format(i, loss.item(), acc.item()))

            
        # Validation - No gradient tracking needed
        with torch.no_grad():

            # Set to evaluation mode
            model.eval()

            # Validation loop
            for j, (inputs, labels) in enumerate(testloader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Forward pass - compute outputs on input data using the model
                outputs = model(inputs)

                # Compute loss
                loss = loss_criterion(outputs, labels)

                # Compute the total loss for the batch and add it to valid_loss
                valid_loss += loss.item() * inputs.size(0)

                # Calculate validation accuracy
                ret, predictions = torch.max(outputs.data, 1)
                correct_counts = predictions.eq(labels.data.view_as(predictions))

                # Convert correct_counts to float and then compute the mean
                acc = torch.mean(correct_counts.type(torch.FloatTensor))

                # Compute total accuracy in the whole batch and add to valid_acc
                valid_acc += acc.item() * inputs.size(0)

                #print("Validation Batch number: {:03d}, Validation: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))
            
        # Find average training loss and training accuracy
        avg_train_loss = train_loss/train_data_size 
        avg_train_acc = train_acc/train_data_size

        # Find average training loss and training accuracy
        avg_test_loss = valid_loss/test_data_size 
        avg_test_acc = valid_acc/test_data_size

        history.append([avg_train_loss, avg_test_loss, avg_train_acc, avg_test_acc])
                
        epoch_end = time.time()
    
        print("Epoch : {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, \n\t\tValidation : Loss : {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(epoch, avg_train_loss, avg_train_acc*100, avg_test_loss, avg_test_acc*100, epoch_end-epoch_start))
        
        # Save if the model has best accuracy till now
        torch.save(model, 'cnn_model_'+str(epoch)+'.pt')
            
    return model, history

In [30]:
num_epochs = 10
trained_model, history = train_and_validate(model, criterion, optimizer, num_epochs)

Epoch: 1/10
Epoch : 000, Training: Loss: 2.3014, Accuracy: 10.8400%, 
		Validation : Loss : 2.2870, Accuracy: 13.0800%, Time: 55.1116s
Epoch: 2/10
Epoch : 001, Training: Loss: 1.0126, Accuracy: 62.3717%, 
		Validation : Loss : 0.5859, Accuracy: 78.0800%, Time: 52.8876s
Epoch: 3/10
Epoch : 002, Training: Loss: 0.5269, Accuracy: 80.1617%, 
		Validation : Loss : 0.5237, Accuracy: 80.3500%, Time: 52.9275s
Epoch: 4/10
Epoch : 003, Training: Loss: 0.4488, Accuracy: 83.2733%, 
		Validation : Loss : 0.4237, Accuracy: 84.7700%, Time: 53.8042s
Epoch: 5/10
Epoch : 004, Training: Loss: 0.3958, Accuracy: 85.3317%, 
		Validation : Loss : 0.4038, Accuracy: 85.4900%, Time: 53.3788s
Epoch: 6/10
Epoch : 005, Training: Loss: 0.3582, Accuracy: 86.8317%, 
		Validation : Loss : 0.3782, Accuracy: 86.1800%, Time: 53.5092s
Epoch: 7/10
Epoch : 006, Training: Loss: 0.3304, Accuracy: 87.8633%, 
		Validation : Loss : 0.3793, Accuracy: 86.2800%, Time: 53.9544s
Epoch: 8/10
Epoch : 007, Training: Loss: 0.3085, Accura

c) Replace your defined CNN in b) with a pre-trained model. Then, proceed with a transfer learning and finetune the model for the Fashion MNIST dataset. **[10 marks]**

In [14]:
model_ft = models.googlenet(pretrained=True)
# print(model_ft)
num_ftrs = model_ft.fc.in_features

model_ft.fc = nn.Linear(num_ftrs,10)

criterion_ft = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft.to(device)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [15]:
num_epochs = 10
trained_model, history = train_and_validate(model_ft, criterion_ft, optimizer_ft, num_epochs)

Epoch: 1/10
Epoch : 000, Training: Loss: 0.3792, Accuracy: 87.0733%, 
		Validation : Loss : 0.2228, Accuracy: 92.0400%, Time: 162.0667s
Epoch: 2/10
Epoch : 001, Training: Loss: 0.2126, Accuracy: 92.3817%, 
		Validation : Loss : 0.1988, Accuracy: 92.9400%, Time: 160.1435s
Epoch: 3/10
Epoch : 002, Training: Loss: 0.1671, Accuracy: 94.1250%, 
		Validation : Loss : 0.1804, Accuracy: 93.7600%, Time: 160.1739s
Epoch: 4/10
Epoch : 003, Training: Loss: 0.1356, Accuracy: 95.1583%, 
		Validation : Loss : 0.1762, Accuracy: 93.8000%, Time: 162.0053s
Epoch: 5/10
Epoch : 004, Training: Loss: 0.1118, Accuracy: 96.0367%, 
		Validation : Loss : 0.1904, Accuracy: 93.4900%, Time: 159.2693s
Epoch: 6/10
Epoch : 005, Training: Loss: 0.0932, Accuracy: 96.6750%, 
		Validation : Loss : 0.1807, Accuracy: 94.1200%, Time: 158.0247s
Epoch: 7/10
Epoch : 006, Training: Loss: 0.0734, Accuracy: 97.4150%, 
		Validation : Loss : 0.2059, Accuracy: 93.7700%, Time: 158.9026s
Epoch: 8/10
Epoch : 007, Training: Loss: 0.0617,

d) Using model-centric methods, propose two (2) strategies that can be used to increase the accuracy of the model on the testing dataset. **[5 marks]**


<span style="color:blue">
    Two model-centric techniques that I propose are: Batch normalization and Dropout</span>

e) Next, implement the two proposed model-centric techniques for the same problem as in the previous question. **[15 marks]**

In [11]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, 3)
        self.batchnorm1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 16, 3)
        self.conv3 = nn.Conv2d(16, 32, 3)
        self.batchnorm2 = nn.BatchNorm2d(32)
        self.conv4 = nn.Conv2d(32,64, 3)
        self.conv5 = nn.Conv2d(64, 128, 3)
        self.batchnorm3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 1 * 1, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 10)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.batchnorm1(self.pool(self.relu(self.conv1(x))))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.batchnorm2(self.pool(self.relu(self.conv3(x))))
        x = self.pool(self.relu(self.conv4(x)))
        x = self.batchnorm3(self.pool(self.relu(self.conv5(x))))
        # print(x.shape)
        x = x.view(-1, 128 * 1 * 1)
        x = self.relu(self.fc1(x))
        x = self.dropout(self.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

In [12]:
model = CNN() # need to instantiate the network to be used in instance method

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)
summary(model, (3, 100, 100))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 98, 98]             224
              ReLU-2            [-1, 8, 98, 98]               0
         MaxPool2d-3            [-1, 8, 49, 49]               0
       BatchNorm2d-4            [-1, 8, 49, 49]              16
            Conv2d-5           [-1, 16, 47, 47]           1,168
              ReLU-6           [-1, 16, 47, 47]               0
         MaxPool2d-7           [-1, 16, 23, 23]               0
            Conv2d-8           [-1, 32, 21, 21]           4,640
              ReLU-9           [-1, 32, 21, 21]               0
        MaxPool2d-10           [-1, 32, 10, 10]               0
      BatchNorm2d-11           [-1, 32, 10, 10]              64
           Conv2d-12             [-1, 64, 8, 8]          18,496
             ReLU-13             [-1, 64, 8, 8]               0
        MaxPool2d-14             [-1, 6

In [13]:
num_epochs = 10
trained_model, history = train_and_validate(model, criterion, optimizer, num_epochs)

Epoch: 1/10
Epoch : 000, Training: Loss: 0.7096, Accuracy: 75.4617%, 
		Validation : Loss : 0.3912, Accuracy: 85.7500%, Time: 55.0822s
Epoch: 2/10
Epoch : 001, Training: Loss: 0.3850, Accuracy: 86.7333%, 
		Validation : Loss : 0.3147, Accuracy: 88.6600%, Time: 54.6635s
Epoch: 3/10
Epoch : 002, Training: Loss: 0.3253, Accuracy: 88.7517%, 
		Validation : Loss : 0.2923, Accuracy: 89.2600%, Time: 54.5742s
Epoch: 4/10
Epoch : 003, Training: Loss: 0.2941, Accuracy: 89.7400%, 
		Validation : Loss : 0.2708, Accuracy: 90.1300%, Time: 55.1209s
Epoch: 5/10
Epoch : 004, Training: Loss: 0.2684, Accuracy: 90.6183%, 
		Validation : Loss : 0.2637, Accuracy: 90.6500%, Time: 54.6985s
Epoch: 6/10
Epoch : 005, Training: Loss: 0.2548, Accuracy: 91.1250%, 
		Validation : Loss : 0.2753, Accuracy: 90.4100%, Time: 55.4079s
Epoch: 7/10
Epoch : 006, Training: Loss: 0.2417, Accuracy: 91.5950%, 
		Validation : Loss : 0.2501, Accuracy: 91.3200%, Time: 54.2054s
Epoch: 8/10
Epoch : 007, Training: Loss: 0.2216, Accura

f) Do you see any accuracy improvement? Whether it is a "yes" or "no", discuss the possible reasons contributing to the accuracy improvement/ unimprovement. **[5 marks]**

<span style="color:blue">
    Yes. Batch normalization standardizes the feature maps to a layer for each mini batch, stabilizing the learning process for the next layer.  Dropout randomly drops units based on predefined ratio on a neural layer, preventing overfitting. However, the changes due to these approaches are not that significant for small network with low number of trainable parameters. </span>

g) In real applications, data-centric strategies are essential to train robust deep learning models. Give two (2) examples of such strategies and discuss how the strategies helps improving the model accuracy. **[5 marks]**

<span style="color:blue">
    Image augmentation or transformation with rotation and dimension flip. The strategies are supposed to improve the data diversity by generating copies of instance variations for training. </span>

h) Next, implement the two proposed data-centric techniques for the same problem as in the previous question. **[10 marks]**

In [20]:
# Applying Transforms to the Data
import torchvision
import torchvision.transforms as transforms

image_transforms = {
    'train': transforms.Compose([
        transforms.Resize((100,100)),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.repeat(3,1,1) if x.size(0)==1 else x),
        transforms.Normalize([0.5, 0.5, 0.5],
                             [0.5, 0.5, 0.5])
    ]),
    'test': transforms.Compose([
        transforms.Resize((100,100)),
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.repeat(3,1,1) if x.size(0)==1 else x),
        transforms.Normalize([0.5, 0.5, 0.5],
                             [0.5, 0.5, 0.5])
    ])
}

In [21]:
# batch_size
batch_size = 16

# datasets
trainset = torchvision.datasets.FashionMNIST('./data',
    download=True,
    train=True,
    transform=image_transforms['train'])
testset = torchvision.datasets.FashionMNIST('./data',
    download=True,
    train=False,
    transform=image_transforms['test'])

# dataloaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                        shuffle=True, num_workers=2)


testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                        shuffle=False, num_workers=2)

# constant for classes
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

train_data_size = len(trainloader.dataset)
test_data_size = len(testloader.dataset)

print(train_data_size)
print(test_data_size)

60000
10000


In [22]:
num_epochs = 10
trained_model, history = train_and_validate(model, criterion, optimizer, num_epochs)

Epoch: 1/10
Epoch : 000, Training: Loss: 0.3668, Accuracy: 87.3033%, 
		Validation : Loss : 0.2586, Accuracy: 90.8500%, Time: 51.9475s
Epoch: 2/10
Epoch : 001, Training: Loss: 0.3134, Accuracy: 89.1117%, 
		Validation : Loss : 0.2546, Accuracy: 90.8500%, Time: 50.8564s
Epoch: 3/10
Epoch : 002, Training: Loss: 0.2965, Accuracy: 89.6250%, 
		Validation : Loss : 0.2636, Accuracy: 90.3700%, Time: 50.8341s
Epoch: 4/10
Epoch : 003, Training: Loss: 0.2823, Accuracy: 90.0633%, 
		Validation : Loss : 0.2493, Accuracy: 91.1700%, Time: 51.9127s
Epoch: 5/10
Epoch : 004, Training: Loss: 0.2832, Accuracy: 90.0283%, 
		Validation : Loss : 0.2414, Accuracy: 91.0700%, Time: 51.1952s
Epoch: 6/10
Epoch : 005, Training: Loss: 0.2704, Accuracy: 90.4683%, 
		Validation : Loss : 0.2466, Accuracy: 90.8800%, Time: 51.2830s
Epoch: 7/10
Epoch : 006, Training: Loss: 0.2618, Accuracy: 90.8650%, 
		Validation : Loss : 0.2431, Accuracy: 91.0800%, Time: 52.6895s
Epoch: 8/10
Epoch : 007, Training: Loss: 0.2585, Accura

**QUESTION 2** **[35 marks]**

Firstly, watch this video:

https://drive.google.com/file/d/1bsypahR7I3f_R3DXkfw_tf0BrbCHxE_O/view?usp=sharing

This video shows an example of masked face recognition where the deep learning model is able to detect and classify your face even when wearing a face mask. Using the end-to-end object detection pipeline that you have learned, develop your own masked face recognition such that the model should recognize your face even on face mask while recognize other persons as "others".

Deliverables for this question are:

- the model file. Change the name to <your_name>.pt file (e.g. hasan.pt).
- a short video (~10 secs) containing your face and your friends faces (for inference).